In [31]:
import geopandas as gpd
import pandas as pd
import folium

In [32]:
geo_data = gpd.read_file('suburb-10-vic.geojson')

In [42]:
# Input text
input_text = """
3139
3211 to 3334
3340 to 3424
3430 to 3649
3658 to 3749
3753
3756
3758
3762
3764
3778 to 3781
3783
3797
3799
3810 to 3909
3921 to 3925
3945 to 3974
3979
3981 to 3996
"""

# Split the input text into lines
input_lines = input_text.strip().split('\n')

# List to hold the processed postcodes
postcodes = []

# Process each line
for line in input_lines:
    # If the line contains the word 'to', it represents a range of postcodes
    if 'to' in line:
        start, end = map(int, line.split(' to '))
        # Extend the postcodes list with the range of postcodes
        postcodes.extend(str(i) for i in range(start, end + 1))
    # Otherwise, it's a single postcode
    else:
        postcodes.append(line)

# Join the processed postcodes with newline characters
output_text = '\n'.join(postcodes)

# Write the output text to a file
with open('postcodes.txt', 'w') as f:
    f.write(output_text)

# Now, 'postcodes.txt' contains each postcode on a new line

In [43]:
# Function to convert txt to csv
import csv

def txt_to_csv(txt_file_path, postcodes_file_path, csv_file_path):
    # Load the postcodes from postcodes.txt into a set for efficient lookup
    with open(postcodes_file_path, 'r') as postcodes_file:
        regional_postcodes = set(postcodes_file.read().splitlines())
    
    # Open the txt file for reading
    with open(txt_file_path, 'r') as txt_file:
        csv_reader = csv.reader(txt_file, delimiter=',')
        headers = next(csv_reader, None)  # Get headers from txt file
        headers.append('Is Regional')  # Add new column header
        
        # Process and filter data
        filtered_data = [headers]  # Start with headers
        for row in csv_reader:
            if row[1].upper() == "VIC":
                # Check if postcode is in regional_postcodes set
                is_regional = row[2] in regional_postcodes
                row.append('True' if is_regional else 'False')
                filtered_data.append(row)
    
    # Write data to CSV file
    with open(csv_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerows(filtered_data)

# File paths
txt_file_path = 'PostcodeData-final.txt'
postcodes_file_path = 'postcodes.txt'
csv_file_path = 'filtered_data.csv'

# Call the function
txt_to_csv(txt_file_path, postcodes_file_path, csv_file_path)


In [44]:

# Load the data from the CSV file
data = pd.read_csv('filtered_data.csv')

# Create a base map
m = folium.Map(location=[-37.8136, 144.9631], zoom_start=10)  # Coordinates for Melbourne, VIC as an example

# Iterate through each row of the data
for index, row in data.iterrows():
    # Get the suburb name, postcode, latitude, and longitude
    suburb = row['Suburb']
    postcode = row['Postcode']
    lat = row['Lat']
    lon = row['Lng']
    is_regional = row['Is Regional']

    # Create a Marker for each suburb with color based on 'Is Regional' column
    color = 'red' if is_regional == 'True' else 'blue'
    icon = folium.Icon(color=color)
    marker = folium.Marker(
        location=[lat, lon],
        tooltip=f'{suburb}, {postcode}, {is_regional}',
        icon=icon
    )

    # Add the Marker to the map
    marker.add_to(m)

# Save the map to an HTML file
folium.GeoJson(
    geo_data,
    name='postcode'
).add_to(m)
folium.LayerControl().add_to(m)
m.save('victoria_map.html')